# Выбор места локации для скважины

# Описание задачи

Имеются пробы нефти в трёх регионах, в каждом из которых по 10000 месторождений, а также замеры качества нефти и информация об объёме её запасов. Необходимо построить модель, позволяющую определить регион, в котором добыча принесёт наибольшую прибыль.

# Условия задачи:

* При разведке региона исследуют 500 точек, из которых выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При текущих ценах один баррель сырья приносит 450 рублей дохода. Поскольку объём указан в тысячах баррелей, то доход с каждой единицы продукта составляет 450 тыс.
* После оценки рисков необходимо оставить только те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирается регион с наибольшей средней прибылью.

# План работы

1. Обзор данных
    1. Загрузка и подготовка данных
2. Обучение и проверка модели
3. Подготовка к расчету прибыли
4. Расчет прибыли и рисков
5. Вывод

# Описание данных

* `id` — уникальный идентификатор скважины;
* `f0`, `f1`, `f2` — три признака точек;
* `product` — объём запасов в скважине (тыс. баррелей).

## Обзор данных

### Загрузка и подготовка данных

In [115]:
# Импорт необходимых библиотек и функций
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [116]:
#Чтение данных для трех регионов.
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [117]:
#Просмотр первых 5 строк таблицы data_0
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [118]:
#Просмотр первых 5 строк таблицы data_1
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [119]:
#Просмотр первых 5 строк таблицы data_2
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [120]:
#Сводная информация о таблице data_0
data_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [121]:
#Сводная информация о таблице data_1
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [122]:
#Сводная информация о таблице data_2
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [123]:
#Удаление столбца с уникальными идентификаторами из таблицы каждого региона.
data_0 = data_0.drop(['id'], axis=1)
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)

In [124]:
#Выделение в таблице первого региона основных признаков и целевого.
features_0 = data_0.drop(['product'], axis=1)
target_0 = data_0['product']

In [125]:
#Выделение в таблице второго региона основных признаков и целевого.
features_1 = data_1.drop(['product'], axis=1)
target_1 = data_1['product']

In [126]:
#Выделение в таблице третьего региона основных признаков и целевого.
features_2 = data_2.drop(['product'], axis=1)
target_2 = data_2['product']

## Обучение и проверка модели

In [127]:
#Разбиение данных первого региона на обучающую и валидационную выборки.
features_0_train, features_0_valid, target_0_train, target_0_valid = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

In [128]:
#Разбиение данных второго региона на обучающую и валидационную выборки.
features_1_train, features_1_valid, target_1_train, target_1_valid = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

In [129]:
#Разбиение данных третьего региона на обучающую и валидационную выборки.
features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

In [130]:
#Создание для каждого региона модели линейной регрессии.
model_0 = LinearRegression()
model_1 = LinearRegression()
model_2 = LinearRegression()

In [131]:
#Обучение каждой модели на данных соответствующих регионов
model_0.fit(features_0_train, target_0_train)
model_1.fit(features_1_train, target_1_train)
model_2.fit(features_2_train, target_2_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [132]:
#Предсказания с использованием модели и валидационной выборки каждого региона.
predictions_0_valid = model_0.predict(features_0_valid)
predictions_1_valid = model_1.predict(features_1_valid)
predictions_2_valid = model_2.predict(features_2_valid)

In [133]:
#Вычисление среднего для спрогнозированных данных в каждом регионе
predictions_0_valid_mean = predictions_0_valid.mean()
predictions_1_valid_mean = predictions_1_valid.mean()
predictions_2_valid_mean = predictions_2_valid.mean()

In [134]:
#Вычисление RMSE для каждого региона
result_0 = mean_squared_error(target_0_valid, predictions_0_valid) ** 0.5
result_1 = mean_squared_error(target_1_valid, predictions_1_valid) ** 0.5
result_2 = mean_squared_error(target_2_valid, predictions_2_valid) ** 0.5

In [135]:
#Вывод вычисленных средних значений для спрогнозированных данных в каждом регионе.
print("Cредний запас предсказанного сырья в первом регионе:", predictions_0_valid_mean)
print("Cредний запас предсказанного сырья во втором регионе:", predictions_1_valid_mean)
print("Cредний запас предсказанного сырья в третьем регионе:", predictions_2_valid_mean)

Cредний запас предсказанного сырья в первом регионе: 92.59256778438038
Cредний запас предсказанного сырья во втором регионе: 68.728546895446
Cредний запас предсказанного сырья в третьем регионе: 94.96504596800489


In [136]:
#Вывод вычисленного RMSE для каждого региона.
print("RMSE модели  для первого региона:", result_0)
print("RMSE модели для второго региона:", result_1)
print("RMSE модели для третьего региона:", result_2)

RMSE модели  для первого региона: 37.5794217150813
RMSE модели для второго региона: 0.893099286775616
RMSE модели для третьего региона: 40.02970873393434


Вывод. Как видно по значениям метрики RMSE для регионов, с большим отрывом, наиболее точной является модель второго региона. Эту модель можно назвать самой точной, она предсказывает целевой параметр с наименьшими ошибками.

## Подготовка к расчёту прибыли

In [137]:
#Бюджет на разработку скважин в регионе, создание константы.
BUDGET = 10000000000

#Доход с каждой единицы продукта, создание константы.
PROFIT = 450000

#200 наиболее перспективных скважин для разработки, создание константы.
TOP_POINTS = 200

In [138]:
#Вычисление достаточного объёма сырья для безубыточной разработки новой скважины.
volume_min = BUDGET / (PROFIT * TOP_POINTS)

In [139]:
#Вывод полученного значения достаточного объёма сырья для безубыточной разработки новой скважины.
print('Достаточный объём сырья для безубыточной разработки новой скважины:', volume_min)

Достаточный объём сырья для безубыточной разработки новой скважины: 111.11111111111111


In [140]:
#Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели.
def profit_oil(subsample, target):
    #Сортировка подвыюорки по убыванию
    subsample_sorted = subsample.sort_values(ascending=False)
    #Выбор 200 лучших скважин
    selected = subsample_sorted[:TOP_POINTS]
    #Суммирование целевого значения объема сырья
    barrel = target[selected.index]
    barrel_sum = barrel.sum()
    #Подсчет прибыли
    sum_income = barrel_sum * PROFIT - BUDGET
    
    return sum_income

Вывод. Как видно из предыдущего пункта, средний запаса предсказанного сырья в каждом регионе для отдельной скважины ниже достаточного объема сырья для безубыточной разработки новой скважины. Наименее рискованной будет разработка скважин в третьем регионе, поскольку средний запас предсказанного сырья в нем больше, чем в остальных регионах.

## Расчёт прибыли и рисков 

In [141]:
#Применение техники Bootstrap с 1000 выборок для каждого региона.

state = np.random.RandomState(12345)
#Преобразование предсказаний в Series
predictions_0_valid = pd.Series(predictions_0_valid, index=target_0_valid.index)
predictions_1_valid = pd.Series(predictions_1_valid, index=target_1_valid.index)
predictions_2_valid = pd.Series(predictions_2_valid, index=target_2_valid.index)
#Создание списков для 1000 значений рассчитанной прибыли тысячи разных подвыборок, в каждом регионе.
values_0 = []
values_1 = []
values_2 = []
#Поочередное взятие 1000 разных подвыборок из предсказанных значений для каждого региона.
for i in range(1000):
    subsample_0 = predictions_0_valid.sample(n=500, replace=False, random_state=state)
    subsample_1 = predictions_1_valid.sample(n=500, replace=False, random_state=state)
    subsample_2 = predictions_2_valid.sample(n=500, replace=False, random_state=state)
    #Вычисление значения прибыли для каждого региона с использованием каждой из 1000 подвыборок 
    values_0.append(profit_oil(subsample_0, target_0))
    values_1.append(profit_oil(subsample_1, target_1))
    values_2.append(profit_oil(subsample_2, target_2))   

In [142]:
#Преобразование списков с возможными значениями прибыли в Series.
values_0 = pd.Series(values_0)
values_1 = pd.Series(values_1)
values_2 = pd.Series(values_2)

In [143]:
#Вычисление среднего значения прибыли для каждого региона.
mean_0 = values_0.mean()
mean_1 = values_1.mean()
mean_2 = values_2.mean()

In [144]:
print('Cреднее значения прибыли для первого региона:', mean_0)
print('Cреднее значения прибыли для второго региона:', mean_1)
print('Cреднее значения прибыли для третьего региона:', mean_2)

Cреднее значения прибыли для первого региона: 383130172.9119874
Cреднее значения прибыли для второго региона: 461647880.15728706
Cреднее значения прибыли для третьего региона: 385875293.1307218


In [145]:
#Функция для вычисления доверительного интервала
def confidence_interval(values):
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    return (lower, upper)

In [147]:
#Вычисление для прибыли 95%-го доверительного интервала, для каждого региона.
confidence_interval_0 = confidence_interval(values_0)
confidence_interval_1 = confidence_interval(values_1)
confidence_interval_2 = confidence_interval(values_2)

In [148]:
print('95%-й доверительный интервал для первого региона:', confidence_interval_0)
print('95%-й доверительный интервал для второго региона::', confidence_interval_1)
print('95%-й доверительный интервал для третьего региона::', confidence_interval_2)

95%-й доверительный интервал для первого региона: (-122628872.20709155, 907659204.8963034)
95%-й доверительный интервал для второго региона:: (70232291.72482277, 843920466.3157415)
95%-й доверительный интервал для третьего региона:: (-125915595.7251412, 924563382.8229748)


In [149]:
#Доля возможных убытков в первом регионе, в процентах.
print('Доля возможных убытков в первом регионе:','{:.0%}'.format((values_0 < 0).mean()))

Доля возможных убытков в первом регионе: 7%


In [150]:
#Доля возможных убытков во втором регионе, в процентах.
values_1[values_1 < 0].count() / len(values_1)
print('Доля возможных убытков во втором регионе:','{:.2%}'.format((values_1 < 0).mean()))

Доля возможных убытков во втором регионе: 0.90%


In [151]:
#Доля возможных убытков во третьем регионе, в процентах.
print('Доля возможных убытков в третьем регионе:','{:.1%}'.format((values_2 < 0).mean()))

Доля возможных убытков в третьем регионе: 8.4%


## Вывод

Для разработки месторождения рекомендуется второй регион. Кроме того, что модель для данного региона является самой точной, среднее значение прибыли для данного региона является самым высоким и вероятность убытков ниже требуемых 2.5%, то есть риски приемлемы в отличие от остальных регионов, где вероятность убытков больше 2.5%. Однако, средний запас предсказанного сырья во втором регионе меньше, чем в остальных и меньше достаточного объёма сырья для безубыточной разработки новой скважины. Объяснить самое высокое значение прибыли во втором регионе можно применением техники Bootstrap при ее расчете, которая дает более достоверные и точные значения, чем просто взятие среднего значения.